<a href="https://colab.research.google.com/github/helmieresearch/helmieresearch/blob/main/Momentum_Ranking_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Install packages

In [1]:
!pip install yfinance

In [2]:
!pip install yahooquery

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 812.4 kB/s eta 0:00:00


In [3]:
!pip install stockstats

###Import data

In [4]:
%matplotlib inline
import datetime
from datetime import date
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import mpld3
#mpld3.enable_notebook()
"""
Settings
"""
portfolio_value = 2000.00
perf_last_month = 2000.00
start = "2020-01-01"
end = "2023-08-04"
ranking_day = "2023-08-02"

# Storing today's date into a variable
today = date.today()
# Storing the specific date
trading_day = today
# Calculate percent difference since last month
perf_pct = (portfolio_value / perf_last_month) - 1
# Print performance, format as percent with two decimals.
print("{} - Last Month Result: {:.2%}".format(trading_day, perf_pct))
# Remember today's portfolio value for next month's calculation
#perf_last_month = portfolio_value

2023-08-02 - Last Month Result: 0.00%


In [5]:

now = datetime.datetime.now().strftime("%Y-%m-%d")
data = yf.Ticker("^J203.JO")
index = data.history(start=start,  end=end)
resetindex = index.reset_index(inplace = True)

index['Date'] = pd.to_datetime(index['Date']).dt.strftime('%Y-%m-%d')

index = index.drop(columns=(['Open', 'High','Low','Volume','Dividends','Stock Splits']))

index.set_index("Date", inplace=True)

index.to_csv('jse_index.csv')

In [7]:
#Import required libraries
import pandas as pd
import json
import requests
import glob
import os
from yahooquery import Ticker
from datetime import datetime

params = {'access_key': 'e52cf3b93696352e880916f8c8adbf0c',
          'limit': 350}
api_result = requests.get('http://api.marketstack.com/v1/exchanges/XJSE/tickers', params)
api_response = api_result.json()
print(f"Exchange Name = {api_response['data']['name']}")
for ticker in api_response['data']['tickers']:
  print(f"{ticker['name']}: {ticker['symbol']}")


# Serializing json
json_object = json.dumps(api_response, indent=4)
# Writing to sample.json
with open("jse.json", "w") as outfile:
    outfile.write(json_object)

# Opening JSON file
with open('jse.json', 'r') as openfile:
    # Reading from json file
    json_object = json.load(openfile)
df1 = pd.json_normalize(json_object["data"]["tickers"])

# dropping null value columns to avoid errors
# new data frame with split value columns
new = df1["symbol"].str.split(".", n = 1, expand = True)
# making separate first name column from new data frame
df1["ticker"]= new[0]
# Dropping old Name columns
df1.drop(columns =["has_intraday","has_eod"], inplace = True)
new.columns = new.columns.astype(str)
new.columns.values[1] = "tickers"
new['0'] + '.' + new['tickers']
new.replace("XJSE", "JO", inplace=True)

result = pd.concat([df1, new], axis=1, join='inner')
df2=result['0'] + '.' + result['tickers']
final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)
#Dropping old Name columns
final.drop(columns =["name", "symbol","ticker"], inplace = True)
# Converting a specific Dataframe
# column to list using Series.tolist()
xjse = final["tickers"].tolist()
print("Converting tickers to list...")
print("Output dataframe..." )
print("Writing csv file..." )
print("Saving JSE data to disc...")
print("Saving csv files for each stock to disc...")
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start= start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)


#Convert dataframe to csv and save to disc
final.to_csv("tickers.csv")
data.to_csv("jse_stocks.csv")

resetindex = data.reset_index(inplace = True)

for i, g in data.groupby('symbol'):
    g.to_csv('/content/Stocks/{}.csv'.format(i), header=True, index_label=False)

# merging the files
joined_files = os.path.join("/content/Stocks/", "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

# new dataframe with different column order
df_new = df.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.to_csv("/content/tickers.csv")

#Change date format to Year-Month-Day
df_new['date'] =  pd.to_datetime(df_new['date']).dt.strftime('%Y-%m-%d')

df_new.set_index('date', inplace=True)

for i, g in df_new.groupby('symbol'):
    g.to_csv('/content/Stocks_final3/{}.csv'.format(i), header=True, index_label=False)

Exchange Name = Johannesburg Stock Exchange
PROSUS N.V.: PRX.XJSE
ANHEUSER-BUSCH INBEV SA NV: ANH.XJSE
BHP GROUP PLC: BHP.XJSE
BRITISH AMERICAN TOB PLC: BTI.XJSE
NASPERS LTD -N-: NPN.XJSE
COMPAGNIE FIN RICHEMONT: CFR.XJSE
GLENCORE PLC: GLN.XJSE
ANGLO AMERICAN PLC: AGL.XJSE
FIRSTRAND LTD: FSR.XJSE
ANGLO AMERICAN PLAT LTD: AMS.XJSE
STANDARD BANK GROUP LTD: SBK.XJSE
VODACOM GROUP LTD: VOD.XJSE
MONDI PLC: MNP.XJSE
CAPITEC BANK HLDGS LTD: CPI.XJSE
BEE - SASOL LIMITED: SOLBE1.XJSE
SANLAM LIMITED: SLM.XJSE
MTN GROUP LTD: MTN.XJSE
ANGLOGOLD ASHANTI LTD: ANG.XJSE
SASOL LIMITED: SOL.XJSE
ABSA GROUP LIMITED: ABG.XJSE
SOUTH32 LIMITED: S32.XJSE
RMB HOLDINGS LTD: RMH.XJSE
IMPALA PLATINUM HLGS LTD: IMP.XJSE
BID CORPORATION LTD: BID.XJSE
REMGRO LTD: REM.XJSE
KUMBA IRON ORE LTD: KIO.XJSE
NEDBANK GROUP LTD: NED.XJSE
SIBANYE STILLWATER LTD: SSW.XJSE
GOLD FIELDS LTD: GFI.XJSE
INVESTEC LTD: INL.XJSE
INVESTEC PLC: INP.XJSE
OLD MUTUAL LIMITED: OMU.XJSE
NEPI ROCKCASTLE PLC: NRP.XJSE
DISCOVERY LTD: DSY.XJSE
SH

In [8]:
symbols = xjse
tickers = Ticker(symbols, asynchronous=True)
data = tickers.history(start=start, end=end, interval='1d')
data= data.drop(['adjclose','dividends','splits'],axis=1)

resetindex = data.reset_index(inplace = True)

# new dataframe with different column order
df_new = data.iloc[:, [1, 2, 3, 4, 5, 6, 0]]

Tickers = df_new["symbol"].tolist()
mylist = list(dict.fromkeys(Tickers))
df = pd.DataFrame (mylist, columns = ['tickers'])
df.columns = df.iloc[1]

df.to_csv("/content/Stocks_final3/tickers.csv")

from pathlib import Path
import pandas as pd
import numpy as np
from scipy import stats

csv_folder = Path('/content/Stocks_final3')

for file in csv_folder.glob('*.JO.csv'): #create iteratable & iterate on it.
    mydata = pd.read_csv(file)
    mydata.reset_index(inplace = True)
    mydata = mydata.rename(columns={'index': 'date'}, index=None)
    mydata['date'] =  pd.to_datetime(mydata['date']).dt.strftime('%Y-%m-%d')
    mydata.set_index('date', inplace=True)
    df =mydata[["open","high","low","close","volume"]]
    no_outlier_prices = df[(np.abs(stats.zscore(df)) <3).all(axis=1)]
    new_file_name = file.parent.joinpath(f"{file.stem}.csv")
    no_outlier_prices.to_csv(new_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

###Ranking list to construct initial portfolio

In [9]:
%matplotlib inline
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (12, 9) # (w, h)
plt.ioff()

tickers = pd.read_csv('/content/Stocks_final3/tickers.csv', header=None)[1].tolist()
stocks = (
    (pd.concat(
        [pd.read_csv(f"/content/Stocks_final3/{ticker}.csv",parse_dates=None, index_col=0)[
        'close'].rename(ticker)
        for ticker in tickers],
        axis=1,
        sort=True)
    )
)
stocks = stocks.loc[:,~stocks.columns.duplicated()]

from scipy.stats import linregress
def momentum(closes):
    returns = np.log(closes)
    x = np.arange(len(returns))
    slope, _, rvalue, _, _ = linregress(x, returns)
    return ((1 + slope) ** 252) * (rvalue ** 2)  # annualize slope and multiply by R^2

momentums = stocks.copy(deep=True)
for ticker in tickers:
    momentums[ticker] = stocks[ticker].rolling(90).apply(momentum, raw=False)

result = momentums.sort_values(by =ranking_day, axis = 1, ascending =False)
result.reset_index(inplace=True)
res=result[~(result['date'] < ranking_day)].dropna(axis=1)

mes = res.reindex(columns = res.columns.tolist()
                                  + ['empty'])

df = mes.set_index(['empty']).stack().reset_index(name='index').rename(columns={'level_2':'date'})

df.columns = df.iloc[0]

df.index = list(df.index)

df.columns.values[0] = "empty"
df.columns.values[1] = "symbol"
df.columns.values[2] = "momentum_score"

df=df.drop(['empty'], axis=1)

finalframe=df.drop(index=0)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

finalframe['rank']=finalframe['momentum_score'].rank(ascending=False)

finalframe['momentum_score'] =finalframe['momentum_score'].astype(float)

#finalframe.style.format({'momentum_score':"{:.2f}"})

Mylist = finalframe["symbol"].tolist()

finalframe.set_index('symbol',inplace=True)

final = pd.concat([df1, df2], axis=1, join='inner')
final.columns.values[3] = "tickers"
final.columns = final.columns.astype(str)

final.set_index('tickers', inplace=True)
Yes = final.reindex(Mylist)

rt = pd.concat([finalframe, Yes], axis=1, join='inner')

rt = rt.iloc[:, [2, 3, 4, 0, 1]]

#rt.style.format({'momentum_score':"{:.2f}"})

# get names of indexes for which column "momentum_score" has value > 40%
index_names = rt[(rt['momentum_score'] < 0.001)].index

# drop these given row indexes from dataFrame
rt.drop(index_names, inplace = True)

rt.reset_index(inplace=True)

My_final_list = rt["index"].tolist()

rt.set_index('index',inplace=True)

final_table_columns = My_final_list
final_table =stocks.drop(columns=[col for col in stocks if col not in final_table_columns])

for column in final_table.columns:
    final_table[column].to_csv('/content/momentum_tickers/' + column + '.csv')

In [10]:
from pathlib import Path
import pandas as pd
import stockstats
from stockstats import StockDataFrame as sdf
import glob
vola_table = Path('/content/momentum_tickers')

for file in vola_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df = pd.read_csv(file)
    vola_df.columns.values[1] = "close"
    vola_df["20d_vol"] = vola_df["close"].pct_change().rolling(20).std()
    #vola_df = sdf.retype(vola_df)['close_90_roc']
    #vola_df = pd.DataFrame(vola_df)
    vola_df['date'] =  pd.to_datetime(vola_df['date']).dt.strftime('%Y-%m-%d')
    vola_df.set_index('date', inplace=True)
    stockstats_df = sdf.retype(vola_df)['close_90_roc']
    df = pd.DataFrame(stockstats_df)
    vola_file_name= file.parent.joinpath(f"/content/ranking_table/{file.stem}.csv")
    vola_df.to_csv(vola_file_name) #, index=None, encoding='utf-8-sig', decimal=',')

vola_table1 = Path('/content/ranking_table')

for file in vola_table1.glob('*.JO.csv'): #create iteratable & iterate on it.
    vola_df1 = pd.read_csv(file).tail(1)

    vola_df1['date'] =  pd.to_datetime(vola_df1['date']).dt.strftime('%Y-%m-%d')

    vola_df1.set_index('date', inplace=True)

    vola_file_name1= file.parent.joinpath(f"/content/momentum_stocks/{file.stem}.csv")

    vola_df1.to_csv(vola_file_name1)


# set search path and glob for files
# here we want to look for csv files in the input directory
path = '/content/'
files = glob.glob(path + '/momentum_stocks/*.csv')

# create empty list to store dataframes
li = []

# loop through list of files and read each one into a dataframe and append to list
for f in files:
    # get filename
    stock = os.path.basename(f)
    # read in csv
    temp_df = pd.read_csv(f)
    # create new column with filename
    temp_df['ticker'] = stock
    # data cleaning to remove the .csv
    temp_df['ticker'] = temp_df['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li.append(temp_df)
    #print(f'Successfully created dataframe for {stock} with shape {temp_df.shape}')

# concatenate our list of dataframes into one!
df = pd.concat(li)

ranking_list =df.set_index('ticker').reindex(My_final_list).reset_index()

inv_vol = 1 / ranking_list["20d_vol"]
sum_inv_vol=inv_vol.sum()
vol_targets = inv_vol / sum_inv_vol
ranking_list["weight"] = vol_targets

ranking_list.set_index('ticker', inplace=True)

final_list = pd.concat([rt,ranking_list], axis=1, join='inner')

stockprice = final_list["close"] / 100
pos_weight = portfolio_value * final_list["weight"]
final_list["units"] = pos_weight / stockprice
final_list["ZAR"] = stockprice * final_list["units"]

In [11]:
from pathlib import Path
import pandas as pd

sma = Path('/content/ranking_table')

for file in sma.glob('*.JO.csv'): #create iteratable & iterate on it.
    sma = pd.read_csv(file)
    sma['date'] =  pd.to_datetime(sma['date']).dt.strftime('%Y-%m-%d')
    sma.set_index('date', inplace=True)

    '''Function'''

    sma['100d_sma'] = sma['close'].rolling(100).mean()

    #sma['index_sma']=index['Close'].rolling(200).mean()

    #sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index'] = index['Close']

    sma['index_sma']=index['Close'].rolling(200).mean()

    sma['trend'] = np.where(sma['close'] > sma['100d_sma'], True, False)

    sma['index_trend']= np.where(sma['index'] > sma['index_sma'], True, False)

    sma_file_name= file.parent.joinpath(f"/content/sma_data/{file.stem}.csv")

    sma.to_csv(sma_file_name)

buy_table = Path('/content/sma_data')

for file in buy_table.glob('*.JO.csv'): #create iteratable & iterate on it.
    buy_list_df = pd.read_csv(file).tail(1)

    buytable = file.parent.joinpath(f"/content/buy_list/{file.stem}.csv")

    buy_list_df.to_csv(buytable)

In [12]:
import pandas as pd
import glob

# set search path and glob for files
# here we want to look for csv files in the input directory
path1 = '/content/'
files1 = glob.glob(path1 + '/buy_list/*.csv')

# create empty list to store dataframes
li1 = []

# loop through list of files and read each one into a dataframe and append to list
for f1 in files1:
    # get filename
    stock1 = os.path.basename(f1)
    # read in csv
    temp_df1 = pd.read_csv(f1)
    # create new column with filename
    temp_df1['ticker'] = stock1
    # data cleaning to remove the .csv
    temp_df1['ticker'] = temp_df1['ticker'].replace('.csv', '', regex=True)
    # append df to list
    li1.append(temp_df1)
    #print(f'Successfully created dataframe for {stock1} with shape {temp_df1.shape}')

# concatenate our list of dataframes into one!
Buy = pd.concat(li1)

buying_list = Buy.set_index('ticker').reindex(My_final_list)#.reset_index()

final_buy_list = pd.concat([final_list, buying_list], axis=1, join='inner')

#define list of columns range
cols = [x for x in range(final_buy_list.shape[1])]

#remove second column in DataFrame
cols.remove(12)
cols.remove(13)
cols.remove(14)
cols.remove(15)
cols.remove(16)#view resulting DataFrame
final_buy_list2 =final_buy_list.iloc[:, cols]
final_buy_list2.info()

final_buy_list2 = final_buy_list2.loc[final_buy_list2.trend, :]

index_names1 = final_buy_list2[(final_buy_list2['close_90_roc'] >= 15)].index

# drop these given row indexes from dataFrame
final_buy_list2.drop(index_names1, inplace = True)
final_buy_list2.to_csv("portfolio_280723.csv")
final_buy_list2

<class 'pandas.core.frame.DataFrame'>
Index: 68 entries, TXT.JO to ISB.JO
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            68 non-null     object 
 1   symbol          68 non-null     object 
 2   ticker          68 non-null     object 
 3   momentum_score  68 non-null     float64
 4   rank            68 non-null     float64
 5   date            68 non-null     object 
 6   close           68 non-null     float64
 7   20d_vol         68 non-null     float64
 8   close_90_roc    67 non-null     float64
 9   weight          68 non-null     float64
 10  units           68 non-null     float64
 11  ZAR             68 non-null     float64
 12  100d_sma        60 non-null     float64
 13  index           68 non-null     float64
 14  index_sma       68 non-null     float64
 15  trend           68 non-null     bool   
 16  index_trend     68 non-null     bool   
dtypes: bool(2), float64(11), object(4

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.061284,3.0,2023-08-02,11159.0,0.013930,10.748313,0.019873,0.356171,39.745150,10418.29,76477.21875,75699.847285,True,True
ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.993953,4.0,2023-08-02,1563.0,0.017014,5.894309,0.016270,2.081890,32.539938,1517.18,76477.21875,75699.847285,True,True
INP.JO,INVESTEC PLC,INP.XJSE,INP,0.979429,5.0,2023-08-02,11172.0,0.013788,9.583129,0.020077,0.359421,40.154557,10503.43,76477.21875,75699.847285,True,True
QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,QFH,0.712352,8.0,2023-08-02,500.0,0.023961,11.111111,0.011553,4.621265,23.106325,455.82,76477.21875,75699.847285,True,True
TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.509101,12.0,2023-08-02,670.0,0.019117,8.064516,0.014481,4.322606,28.961460,648.80,76477.21875,75699.847285,True,True
LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.360797,22.0,2023-08-02,2033.0,0.010218,7.851459,0.027091,2.665109,54.181661,2006.54,76477.21875,75699.847285,True,True
MTH.JO,MOTUS HOLDINGS LTD,MTH.XJSE,MTH,0.316870,27.0,2023-08-02,10251.0,0.023525,7.633347,0.011767,0.229585,23.534757,9669.03,76477.21875,75699.847285,True,True
DCP.JO,DIS-CHEM PHARMACIES LTD,DCP.XJSE,DCP,0.266942,30.0,2023-08-02,2560.0,0.022672,2.852551,0.012210,0.953901,24.419867,2461.71,76477.21875,75699.847285,True,True
RDF.JO,REDEFINE PROPERTIES LTD,RDF.XJSE,RDF,0.230312,32.0,2023-08-02,355.0,0.016603,-0.560224,0.016673,9.393517,33.346984,348.58,76477.21875,75699.847285,True,True
AFH.JO,ALEXANDER FORBES GRP HLDGS,AFH.XJSE,AFH,0.178606,38.0,2023-08-02,520.0,0.045662,9.704641,0.006062,2.331680,12.124737,519.58,76477.21875,75699.847285,True,True


In [30]:
final_buy_list

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
TXT.JO,TEXTAINER GROUP HLDGS LTD,TXT.XJSE,TXT,1.606527,1.0,2023-08-02,77416.0,0.020407,32.595701,0.013565,...,897,2023-08-02,77416.0,0.020407,32.595701,67843.71,76477.21875,75699.847285,True,True
HCI.JO,HOSKEN CONS INV LTD,HCI.XJSE,HCI,1.229048,2.0,2023-08-02,22650.0,0.011731,21.110042,0.023597,...,897,2023-08-02,22650.0,0.011731,21.110042,20623.20,76477.21875,75699.847285,True,True
INL.JO,INVESTEC LTD,INL.XJSE,INL,1.061284,3.0,2023-08-02,11159.0,0.013930,10.748313,0.019873,...,897,2023-08-02,11159.0,0.013930,10.748313,10418.29,76477.21875,75699.847285,True,True
ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.993953,4.0,2023-08-02,1563.0,0.017014,5.894309,0.016270,...,897,2023-08-02,1563.0,0.017014,5.894309,1517.18,76477.21875,75699.847285,True,True
INP.JO,INVESTEC PLC,INP.XJSE,INP,0.979429,5.0,2023-08-02,11172.0,0.013788,9.583129,0.020077,...,897,2023-08-02,11172.0,0.013788,9.583129,10503.43,76477.21875,75699.847285,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTNZF.JO,MTN ZAKHELE FUTHI,MTNZF.XJSE,MTNZF,0.007349,64.0,2023-08-02,2050.0,0.027704,12.637363,0.009992,...,897,2023-08-02,2050.0,0.027704,12.637363,1891.81,76477.21875,75699.847285,True,True
MST.JO,MUSTEK LTD,MST.XJSE,MST,0.006197,65.0,2023-08-02,1551.0,0.016880,-5.369128,0.016399,...,897,2023-08-02,1551.0,0.016880,-5.369128,1680.68,76477.21875,75699.847285,False,True
SYG.JO,SYGNIA LIMITED,SYG.XJSE,SYG,0.004920,66.0,2023-08-02,1850.0,0.028391,5.113636,0.009750,...,897,2023-08-02,1850.0,0.028391,5.113636,1725.95,76477.21875,75699.847285,True,True
ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.003168,67.0,2023-08-02,18120.0,0.014389,0.834725,0.019238,...,897,2023-08-02,18120.0,0.014389,0.834725,17197.60,76477.21875,75699.847285,True,True


###List to Liquidate current positions

In [26]:

sell_list = final_buy_list [~final_buy_list
 ["trend"]]
sell_list.info()
dqs = sell_list ["ticker"].tolist()
cpf=pd.read_csv("portfolio_140723.csv")
sell_list

<class 'pandas.core.frame.DataFrame'>
Index: 40 entries, CML.JO to ISB.JO
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            40 non-null     object 
 1   symbol          40 non-null     object 
 2   ticker          40 non-null     object 
 3   momentum_score  40 non-null     float64
 4   rank            40 non-null     float64
 5   date            40 non-null     object 
 6   close           40 non-null     float64
 7   20d_vol         40 non-null     float64
 8   close_90_roc    39 non-null     float64
 9   weight          40 non-null     float64
 10  units           40 non-null     float64
 11  ZAR             40 non-null     float64
 12  Unnamed: 0      40 non-null     int64  
 13  date            40 non-null     object 
 14  close           40 non-null     float64
 15  20d_vol         40 non-null     float64
 16  close_90_roc    39 non-null     float64
 17  100d_sma        32 non-null     f

,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,...,Unnamed: 0,date,close,20d_vol,close_90_roc,100d_sma,index,index_sma,trend,index_trend
CML.JO,CORONATION FUND MNGRS LD,CML.XJSE,CML,0.935294,6.0,2023-08-02,3233.0,0.014409,5.653595,0.019212,...,897,2023-08-02,3233.0,0.014409,5.653595,NaN,76477.21875,75699.847285,False,True
TRU.JO,TRUWORTHS INT LTD,TRU.XJSE,TRU,0.621159,10.0,2023-08-02,6848.0,0.020035,32.507740,0.013817,...,897,2023-08-02,6848.0,0.020035,32.507740,NaN,76477.21875,75699.847285,False,True
JSE.JO,JSE LTD,JSE.XJSE,JSE,0.478341,13.0,2023-08-02,9170.0,0.011482,-12.206798,0.024109,...,897,2023-08-02,9170.0,0.011482,-12.206798,NaN,76477.21875,75699.847285,False,True
BWN.JO,BALWIN PROPERTIES LTD,BWN.XJSE,BWN,0.477492,14.0,2023-08-02,250.0,0.008783,-13.194444,0.031517,...,897,2023-08-02,250.0,0.008783,-13.194444,278.85,76477.21875,75699.847285,False,True
ARI.JO,AFRICAN RAINBOW MIN LTD,ARI.XJSE,ARI,0.418018,15.0,2023-08-02,18911.0,0.021275,-22.941200,0.013011,...,897,2023-08-02,18911.0,0.021275,-22.941200,21905.94,76477.21875,75699.847285,False,True
APF.JO,ACCELERATE PROP FUND LTD,APF.XJSE,APF,0.388135,16.0,2023-08-02,85.0,0.046382,-3.409091,0.005968,...,897,2023-08-02,85.0,0.046382,-3.409091,88.94,76477.21875,75699.847285,False,True
GPL.JO,GRAND PARADE INV LTD,GPL.XJSE,GPL,0.385963,17.0,2023-08-02,298.0,0.021044,-10.510511,0.013155,...,897,2023-08-02,298.0,0.021044,-10.510511,317.35,76477.21875,75699.847285,False,True
HLM.JO,HULAMIN LTD,HLM.XJSE,HLM,0.384462,18.0,2023-08-02,265.0,0.021979,-6.690141,0.012595,...,897,2023-08-02,265.0,0.021979,-6.690141,NaN,76477.21875,75699.847285,False,True
YRK.JO,YORK TIMBER HOLDINGS LTD,YRK.XJSE,YRK,0.374405,19.0,2023-08-02,190.0,0.029044,-11.627907,0.009531,...,897,2023-08-02,190.0,0.029044,-11.627907,201.91,76477.21875,75699.847285,False,True
BAW.JO,BARLOWORLD LTD,BAW.XJSE,BAW,0.366251,20.0,2023-08-02,8464.0,0.013516,-1.271434,0.020481,...,897,2023-08-02,8464.0,0.013516,-1.271434,NaN,76477.21875,75699.847285,False,True


In [21]:
liquidate = cpf.loc[cpf['ticker'].isin (dqs)].reset_index (drop=True)
liquidate.info()
liquidate

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      6 non-null      object 
 1   name            6 non-null      object 
 2   symbol          6 non-null      object 
 3   ticker          6 non-null      object 
 4   momentum_score  6 non-null      float64
 5   rank            6 non-null      float64
 6   date            6 non-null      object 
 7   close           6 non-null      float64
 8   20d_vol         6 non-null      float64
 9   close_90_roc    6 non-null      float64
 10  weight          6 non-null      float64
 11  units           6 non-null      float64
 12  ZAR             6 non-null      float64
 13  100d_sma        6 non-null      float64
 14  index           6 non-null      float64
 15  index_sma       6 non-null      float64
 16  trend           6 non-null      bool   
 17  index_trend     6 non-null      bool   

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,PPH.JO,PEPKOR HOLDINGS LTD,PPH.XJSE,PPH,0.232722,37.0,2023-07-14,1695.0,0.016413,-6.714364,0.014750,1.740358,29.499060,1679.27,77750.507812,74859.902324,True,True
1,ASC.JO,ASCENDIS HEALTH LTD,ASC.XJSE,ASC,0.223080,38.0,2023-07-14,64.0,0.030166,8.474576,0.008025,25.078702,16.050370,62.34,77750.507812,74859.902324,True,True
2,SUR.JO,SPUR CORPORATION LTD,SUR.XJSE,SUR,0.173518,46.0,2023-07-14,2360.0,0.008799,-0.631579,0.027513,2.331587,55.025458,2339.81,77750.507812,74859.902324,True,True
3,4SI.JO,4Sight Holdings Limited,4SI.XJSE,4SI,0.097897,54.0,2023-07-14,28.0,0.040222,-3.448276,0.006019,42.991592,12.037646,26.96,77750.507812,74859.902324,True,True
4,MFL.JO,METROFILE HOLDINGS LTD,MFL.XJSE,MFL,0.039159,63.0,2023-07-14,325.0,0.066643,5.177994,0.003633,2.235468,7.265271,313.42,77750.507812,74859.902324,True,True
5,JBL.JO,JUBILEE METALS GROUP PLC,JBL.XJSE,JBL,0.015929,70.0,2023-07-14,193.0,0.054717,-8.530806,0.004424,4.584854,8.848768,191.44,77750.507812,74859.902324,True,True


In [22]:
newpf = cpf[~cpf['ticker'].isin(liquidate['ticker'])]
newpf .info()
newpf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      21 non-null     object 
 1   name            21 non-null     object 
 2   symbol          21 non-null     object 
 3   ticker          21 non-null     object 
 4   momentum_score  21 non-null     float64
 5   rank            21 non-null     float64
 6   date            21 non-null     object 
 7   close           21 non-null     float64
 8   20d_vol         21 non-null     float64
 9   close_90_roc    21 non-null     float64
 10  weight          21 non-null     float64
 11  units           21 non-null     float64
 12  ZAR             21 non-null     float64
 13  100d_sma        21 non-null     float64
 14  index           21 non-null     float64
 15  index_sma       21 non-null     float64
 16  trend           21 non-null     bool   
 17  index_trend     21 non-null     bool 

,Unnamed: 0,name,symbol,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
0,VUN.JO,VUNANI LTD,VUN.XJSE,VUN,59.892215,1.0,2023-07-14,260.0,0.018176,-10.344828,0.013319,10.245601,26.638563,236.1500,77750.507812,74859.902324,True,True
1,SEB.JO,SEBATA HOLDINGS LTD,SEB.XJSE,SEB,3.075193,2.0,2023-07-14,220.0,0.109355,-12.000000,0.002214,2.012527,4.427560,188.4971,77750.507812,74859.902324,True,True
2,SRE.JO,SIRIUS REAL ESTATE LTD,SRE.XJSE,SRE,1.544453,4.0,2023-07-14,1972.0,0.019070,9.799555,0.012695,1.287476,25.389029,1870.5900,77750.507812,74859.902324,True,True
3,TRE.JO,TRENCOR LTD,TRE.XJSE,TRE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.8500,77750.507812,74859.902324,True,True
4,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,ART,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.8500,77750.507812,74859.902324,True,True
5,REM.JO,REMGRO LTD,REM.XJSE,REM,0.606838,9.0,2023-07-14,15300.0,0.011028,10.365722,0.021952,0.286955,43.904071,13924.5900,77750.507812,74859.902324,True,True
6,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,LHC,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.7600,77750.507812,74859.902324,True,True
7,RLO.JO,REUNERT LTD,RLO.XJSE,RLO,0.274446,34.0,2023-07-14,5947.0,0.015125,0.337439,0.016006,0.538298,32.012604,5914.1000,77750.507812,74859.902324,True,True
8,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,ABG,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.3800,77750.507812,74859.902324,True,True
11,CLI.JO,CLIENTELE LTD,CLI.XJSE,CLI,0.218270,39.0,2023-07-14,1125.0,0.057045,8.173077,0.004244,0.754453,8.487591,1045.9800,77750.507812,74859.902324,True,True


In [24]:
musthave = final_buy_list2 ["ticker"].tolist()
musthave

['INL',
 'ART',
 'INP',
 'QFH',
 'TRE',
 'LHC',
 'MTH',
 'DCP',
 'RDF',
 'AFH',
 'SPG',
 'MTM',
 'NPN',
 'PRX',
 'GNDP',
 'SSK',
 'VKE',
 'ARL',
 'SDO',
 'MTNZF',
 'SYG',
 'ABG']

In [47]:
dfnew = newpf[~newpf['ticker'].isin(final_buy_list2['ticker'])]
dfnew .info()
remove = dfnew ["ticker"].tolist()
remove

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 26
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      13 non-null     object 
 1   name            13 non-null     object 
 2   symbol          13 non-null     object 
 3   ticker          13 non-null     object 
 4   momentum_score  13 non-null     float64
 5   rank            13 non-null     float64
 6   date            13 non-null     object 
 7   close           13 non-null     float64
 8   20d_vol         13 non-null     float64
 9   close_90_roc    13 non-null     float64
 10  weight          13 non-null     float64
 11  units           13 non-null     float64
 12  ZAR             13 non-null     float64
 13  100d_sma        13 non-null     float64
 14  index           13 non-null     float64
 15  index_sma       13 non-null     float64
 16  trend           13 non-null     bool   
 17  index_trend     13 non-null     bool 

['VUN',
 'SEB',
 'SRE',
 'REM',
 'RLO',
 'CLI',
 'HDC',
 'NWL',
 'RBX',
 'DSBP',
 'BVT',
 'ENX',
 'SNT']

In [125]:
now = newpf[~newpf['ticker'].isin(dfnew['ticker'])]
now.set_index("ticker", inplace=True)
now.info()
rebalance = now ["symbol"].tolist()
now

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, TRE to PRX
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8 non-null      object 
 1   name            8 non-null      object 
 2   symbol          8 non-null      object 
 3   momentum_score  8 non-null      float64
 4   rank            8 non-null      float64
 5   date            8 non-null      object 
 6   close           8 non-null      float64
 7   20d_vol         8 non-null      float64
 8   close_90_roc    8 non-null      float64
 9   weight          8 non-null      float64
 10  units           8 non-null      float64
 11  ZAR             8 non-null      float64
 12  100d_sma        8 non-null      float64
 13  index           8 non-null      float64
 14  index_sma       8 non-null      float64
 15  trend           8 non-null      bool   
 16  index_trend     8 non-null      bool   
dtypes: bool(2), float64(11), object(4)
memor

,Unnamed: 0,name,symbol,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
ticker,,,,,,,,,,,,,,,,,
TRE,TRE.JO,TRENCOR LTD,TRE.XJSE,0.923557,7.0,2023-07-14,670.0,0.035485,8.064516,0.006822,2.036480,13.644417,641.85,77750.507812,74859.902324,True,True
ART,ART.JO,ARGENT INDUSTRIAL LTD,ART.XJSE,0.613898,8.0,2023-07-14,1632.0,0.012478,9.898990,0.019402,2.377644,38.803154,1502.85,77750.507812,74859.902324,True,True
LHC,LHC.JO,LIFE HEALTHC GRP HLDGS LTD,LHC.XJSE,0.439935,16.0,2023-07-14,2049.0,0.010084,5.673027,0.024007,2.343246,48.013120,1989.76,77750.507812,74859.902324,True,True
ABG,ABG.JO,ABSA GROUP LIMITED,ABG.XJSE,0.271652,35.0,2023-07-14,17804.0,0.012469,-10.420126,0.019416,0.218105,38.831346,17379.38,77750.507812,74859.902324,True,True
QFH,QFH.JO,QUANTUM FOODS HLDGS LTD,QFH.XJSE,0.144772,50.0,2023-07-14,481.0,0.027969,1.263158,0.008656,3.599027,17.311322,452.61,77750.507812,74859.902324,True,True
SPG,SPG.JO,SUPER GROUP LTD,SPG.XJSE,0.072090,55.0,2023-07-14,3284.0,0.009642,-7.778714,0.025107,1.529037,50.213560,3215.27,77750.507812,74859.902324,True,True
MTM,MTM.JO,MOMENTUM MET HLDGS LTD,MTM.XJSE,0.034560,65.0,2023-07-14,1857.0,0.012406,-5.399898,0.019514,2.101670,39.028021,1816.35,77750.507812,74859.902324,True,True
PRX,PRX.JO,PROSUS N.V.,PRX.XJSE,0.015062,71.0,2023-07-14,140786.0,0.022595,5.189779,0.010714,0.015220,21.428077,135136.25,77750.507812,74859.902324,True,True


In [133]:
portfolio_value2=2076.60

final_weights = final_buy_list
final_weights.set_index('rank').reset_index()
final_w = [x for x in range(final_weights.shape[1])]

final_w.remove(12)
final_w.remove(13)
final_w.remove(14)
final_w.remove(15)
final_w.remove(16)
final_weights2=final_weights.iloc[:, cols]
inv_vol2 = 1 / final_weights2["20d_vol"]
sum_inv_vol2 =inv_vol2.sum()
vol_targets2 = inv_vol2 / sum_inv_vol2
final_weights2["weight"] = vol_targets2

final_weights2.set_index('symbol', inplace=True)

stockprice2 = final_weights2["close"] / 100
pos_weight2 = portfolio_value2 * final_weights2["weight"]
final_weights2["units"] = pos_weight2 / stockprice2
final_weights2["ZAR"] = stockprice2 * final_weights2["units"]
final_weights2

<ipython-input-133-43b7308cee05>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_weights2["weight"] = vol_targets2
<ipython-input-133-43b7308cee05>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_weights2["units"] = pos_weight2 / stockprice2
<ipython-input-133-43b7308cee05>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

,name,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
symbol,,,,,,,,,,,,,,,,
TXT.XJSE,TEXTAINER GROUP HLDGS LTD,TXT,1.606527,1.0,2023-08-02,77416.0,0.020407,32.595701,0.013565,0.036387,28.169152,67843.71,76477.21875,75699.847285,True,True
HCI.XJSE,HOSKEN CONS INV LTD,HCI,1.229048,2.0,2023-08-02,22650.0,0.011731,21.110042,0.023597,0.216338,49.000577,20623.20,76477.21875,75699.847285,True,True
INL.XJSE,INVESTEC LTD,INL,1.061284,3.0,2023-08-02,11159.0,0.013930,10.748313,0.019873,0.369813,41.267389,10418.29,76477.21875,75699.847285,True,True
ART.XJSE,ARGENT INDUSTRIAL LTD,ART,0.993953,4.0,2023-08-02,1563.0,0.017014,5.894309,0.016270,2.161626,33.786218,1517.18,76477.21875,75699.847285,True,True
INP.XJSE,INVESTEC PLC,INP,0.979429,5.0,2023-08-02,11172.0,0.013788,9.583129,0.020077,0.373187,41.692476,10503.43,76477.21875,75699.847285,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MTNZF.XJSE,MTN ZAKHELE FUTHI,MTNZF,0.007349,64.0,2023-08-02,2050.0,0.027704,12.637363,0.009992,1.012194,20.749973,1891.81,76477.21875,75699.847285,True,True
MST.XJSE,MUSTEK LTD,MST,0.006197,65.0,2023-08-02,1551.0,0.016880,-5.369128,0.016399,2.195656,34.054628,1680.68,76477.21875,75699.847285,False,True
SYG.XJSE,SYGNIA LIMITED,SYG,0.004920,66.0,2023-08-02,1850.0,0.028391,5.113636,0.009750,1.094473,20.247751,1725.95,76477.21875,75699.847285,True,True


In [134]:
rebalance1 =final_weights2[final_weights2['name'].isin(now['name'])]
rebalance1 .info()
rebalance1

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, ART.XJSE to ABG.XJSE
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            8 non-null      object 
 1   ticker          8 non-null      object 
 2   momentum_score  8 non-null      float64
 3   rank            8 non-null      float64
 4   date            8 non-null      object 
 5   close           8 non-null      float64
 6   20d_vol         8 non-null      float64
 7   close_90_roc    8 non-null      float64
 8   weight          8 non-null      float64
 9   units           8 non-null      float64
 10  ZAR             8 non-null      float64
 11  100d_sma        8 non-null      float64
 12  index           8 non-null      float64
 13  index_sma       8 non-null      float64
 14  trend           8 non-null      bool   
 15  index_trend     8 non-null      bool   
dtypes: bool(2), float64(11), object(3)
memory usage: 976.0+ bytes


,name,ticker,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend
symbol,,,,,,,,,,,,,,,,
ART.XJSE,ARGENT INDUSTRIAL LTD,ART,0.993953,4.0,2023-08-02,1563.0,0.017014,5.894309,0.016270,2.161626,33.786218,1517.18,76477.21875,75699.847285,True,True
QFH.XJSE,QUANTUM FOODS HLDGS LTD,QFH,0.712352,8.0,2023-08-02,500.0,0.023961,11.111111,0.011553,4.798259,23.991297,455.82,76477.21875,75699.847285,True,True
TRE.XJSE,TRENCOR LTD,TRE,0.509101,12.0,2023-08-02,670.0,0.019117,8.064516,0.014481,4.488162,30.070684,648.80,76477.21875,75699.847285,True,True
LHC.XJSE,LIFE HEALTHC GRP HLDGS LTD,LHC,0.360797,22.0,2023-08-02,2033.0,0.010218,7.851459,0.027091,2.767182,56.256819,2006.54,76477.21875,75699.847285,True,True
SPG.XJSE,SUPER GROUP LTD,SPG,0.148037,40.0,2023-08-02,3429.0,0.018198,2.572540,0.015211,0.921206,31.588146,3219.46,76477.21875,75699.847285,True,True
MTM.XJSE,MOMENTUM MET HLDGS LTD,MTM,0.087199,43.0,2023-08-02,1831.0,0.011084,1.496674,0.024974,2.832374,51.860760,1802.74,76477.21875,75699.847285,True,True
PRX.XJSE,PROSUS N.V.,PRX,0.030591,52.0,2023-08-02,138704.0,0.020456,4.132914,0.013532,0.020260,28.101266,135346.19,76477.21875,75699.847285,True,True
ABG.XJSE,ABSA GROUP LIMITED,ABG,0.003168,67.0,2023-08-02,18120.0,0.014389,0.834725,0.019238,0.220474,39.949951,17197.60,76477.21875,75699.847285,True,True


In [135]:
rebalance3 = rebalance1.reindex(rebalance).set_index('ticker')
rebalance3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, TRE to PRX
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            8 non-null      object 
 1   momentum_score  8 non-null      float64
 2   rank            8 non-null      float64
 3   date            8 non-null      object 
 4   close           8 non-null      float64
 5   20d_vol         8 non-null      float64
 6   close_90_roc    8 non-null      float64
 7   weight          8 non-null      float64
 8   units           8 non-null      float64
 9   ZAR             8 non-null      float64
 10  100d_sma        8 non-null      float64
 11  index           8 non-null      float64
 12  index_sma       8 non-null      float64
 13  trend           8 non-null      bool   
 14  index_trend     8 non-null      bool   
dtypes: bool(2), float64(11), object(2)
memory usage: 912.0+ bytes


In [136]:
now.reset_index(inplace=True)
rebalance3.reset_index(inplace=True)

In [137]:
rebalance3 ["bs"] = rebalance3["units"]-now["units"]

In [138]:
rebalance3

,ticker,name,momentum_score,rank,date,close,20d_vol,close_90_roc,weight,units,ZAR,100d_sma,index,index_sma,trend,index_trend,bs
0,TRE,TRENCOR LTD,0.509101,12.0,2023-08-02,670.0,0.019117,8.064516,0.014481,4.488162,30.070684,648.80,76477.21875,75699.847285,True,True,2.451682
1,ART,ARGENT INDUSTRIAL LTD,0.993953,4.0,2023-08-02,1563.0,0.017014,5.894309,0.016270,2.161626,33.786218,1517.18,76477.21875,75699.847285,True,True,-0.216018
2,LHC,LIFE HEALTHC GRP HLDGS LTD,0.360797,22.0,2023-08-02,2033.0,0.010218,7.851459,0.027091,2.767182,56.256819,2006.54,76477.21875,75699.847285,True,True,0.423936
3,ABG,ABSA GROUP LIMITED,0.003168,67.0,2023-08-02,18120.0,0.014389,0.834725,0.019238,0.220474,39.949951,17197.60,76477.21875,75699.847285,True,True,0.002370
4,QFH,QUANTUM FOODS HLDGS LTD,0.712352,8.0,2023-08-02,500.0,0.023961,11.111111,0.011553,4.798259,23.991297,455.82,76477.21875,75699.847285,True,True,1.199232
5,SPG,SUPER GROUP LTD,0.148037,40.0,2023-08-02,3429.0,0.018198,2.572540,0.015211,0.921206,31.588146,3219.46,76477.21875,75699.847285,True,True,-0.607831
6,MTM,MOMENTUM MET HLDGS LTD,0.087199,43.0,2023-08-02,1831.0,0.011084,1.496674,0.024974,2.832374,51.860760,1802.74,76477.21875,75699.847285,True,True,0.730703
7,PRX,PROSUS N.V.,0.030591,52.0,2023-08-02,138704.0,0.020456,4.132914,0.013532,0.020260,28.101266,135346.19,76477.21875,75699.847285,True,True,0.005040
